# E-commerce Analytics — Relatório Analítico
Time de Dados — Projeto de Inferência & KPI para E-commerce Brasileiro

**Instruções**: Preencha o diretório `data/` com CSVs correspondentes às tabelas (DIM_* e FACT_Orders) e rode as células. O notebook gera tabelas, testes, gráficos e salva figuras em `figures/`.


In [3]:
# Imports principais
import os, math, itertools, warnings
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import duckdb
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 120)
BASE = 'C:/ecommerce_analytics_project/ecommerce_analytics_project'

## 1) Conexão & Carregamento (DuckDB + CSVs)

In [4]:
# Conecte ao DuckDB e crie/valide o esquema
con = duckdb.connect(os.path.join(BASE, 'warehouse.duckdb'))
con.execute(open(os.path.join(BASE, 'sql', 'schema.sql'), 'r', encoding='utf-8').read())

# Carregar CSVs se existirem (AUTO_DETECT)
csvs = {
    'DIM_Customer': 'DIM_Customer.csv',
    'DIM_Delivery': 'DIM_Delivery.csv',
    'DIM_Products': 'DIM_Products.csv',
    'DIM_Shopping': 'DIM_Shopping.csv',
    'FACT_Orders': 'FACT_Orders.csv'
}
for table, fname in csvs.items():
    fpath = os.path.join(BASE, 'data', fname)
    if os.path.exists(fpath):
        con.execute(f"COPY {table} FROM '{fpath}' (AUTO_DETECT TRUE, HEADER TRUE)")

In [5]:
# Criar view de features e alguns KPIs base
con.execute(open(os.path.join(BASE, 'sql', 'analysis.sql'), 'r', encoding='utf-8').read())
con.execute('SELECT * FROM v_orders LIMIT 5').df()

,Id,Order_Date,Discount,Subtotal,Total,payment,Purchase_Status,Delivery_Id,Customer_Id,Services,P_Sevice,D_Forecast,D_Date,delivery_delay_days,delivery_lead_time,is_late,is_confirmed,freight_share,discount_abs
0,1,2024-02-07,0.15,4177.23,3605.36,PIX,Confirmado,D0365,C0046,Standard,15.67,2024-10-12,2024-10-12,0,248,0,1,0.004346,626.5845
1,2,2024-06-09,0.05,3569.02,3403.52,Débito,Confirmado,D0188,C0140,Standard,30.82,2024-02-17,2024-02-18,1,-112,1,1,0.009055,178.4510
2,3,2024-01-03,0.15,3973.87,3425.00,Boleto,Confirmado,D0034,C0036,Standard,27.62,2024-07-21,2024-07-20,-1,199,0,1,0.008064,596.0805
3,4,2024-02-07,0.05,4319.17,4137.94,PIX,Confirmado,D0406,C0193,Same-Day,51.64,2024-02-28,2024-02-29,1,22,1,1,0.012480,215.9585
4,5,2024-09-29,0.05,4412.59,4219.50,PIX,Confirmado,D0489,C0271,Scheduled,62.69,2024-04-17,2024-04-15,-2,-167,0,1,0.014857,220.6295


## 2) Data Quality & Preparação

In [7]:
# Checagem de chaves e unicidade
dq = {}
dq['orders_vs_distinct'] = con.execute('SELECT COUNT(*) AS total, COUNT(DISTINCT Id) AS distinct_count FROM FACT_Orders').df()
dq['orphan_customers'] = con.execute("""
SELECT COUNT(*) AS orphan_customers
FROM FACT_Orders f
LEFT JOIN DIM_Customer c ON f.Customer_Id = c.Customer_Id
WHERE c.Customer_Id IS NULL""").df()
dq

{'orders_vs_distinct':    total  distinct_count
 0    500             500,
 'orphan_customers':    orphan_customers
 0                 0}

## 3) EDA (hist, box, correlação, séries)

In [8]:
# Carregar v_orders como DataFrame para EDA
v_orders = con.execute('SELECT * FROM v_orders').df()
# Conversões defensivas
numeric_cols = ['Total','Subtotal','Discount','P_Sevice','delivery_delay_days','delivery_lead_time','freight_share','discount_abs']
for c in numeric_cols:
    if c in v_orders.columns:
        v_orders[c] = pd.to_numeric(v_orders[c], errors='coerce')

# Histogramas (matplotlib, 1 gráfico por vez, sem estilos/cores explícitas)
for col in ['Total','delivery_lead_time','Discount']:
    if col in v_orders.columns:
        plt.figure()
        v_orders[col].dropna().plot(kind='hist', bins=30, title=f'Histograma — {col}')
        plt.xlabel(col)
        plt.ylabel('Frequência')
        out = os.path.join(BASE, 'figures', f'hist_{col}.png')
        plt.savefig(out, bbox_inches='tight')
        plt.close()

# Boxplots
for col in ['Total','delivery_lead_time']:
    if col in v_orders.columns:
        plt.figure()
        v_orders[[col]].boxplot()
        plt.title(f'Boxplot — {col}')
        out = os.path.join(BASE, 'figures', f'box_{col}.png')
        plt.savefig(out, bbox_inches='tight')
        plt.close()

# Correlação (Pearson)
corr_cols = [c for c in numeric_cols if c in v_orders.columns]
if corr_cols:
    corr = v_orders[corr_cols].corr(method='pearson')
    corr.to_csv(os.path.join(BASE, 'figures', 'correlations.csv'), index=True)
corr.head() if corr_cols else None


,Total,Subtotal,Discount,P_Sevice,delivery_delay_days,delivery_lead_time,freight_share,discount_abs
Total,1.000000,0.992223,-0.133407,0.037672,0.048170,0.007712,-0.519550,0.407640
Subtotal,0.992223,1.000000,-0.025576,0.040552,0.050592,-0.002522,-0.521884,0.517747
Discount,-0.133407,-0.025576,1.000000,-0.006048,0.060683,-0.085246,0.057575,0.732839
P_Sevice,0.037672,0.040552,-0.006048,1.000000,0.108722,0.004094,0.278930,0.041276
delivery_delay_days,0.048170,0.050592,0.060683,0.108722,1.000000,0.006260,-0.037610,0.042524


## 4) Inferência — ICs (95%)

In [9]:
def ci_mean(series, alpha=0.05):
    x = series.dropna().astype(float)
    n = len(x)
    if n < 2:
        return (np.nan, np.nan, np.nan, n)
    m = x.mean()
    s = x.std(ddof=1)
    tcrit = stats.t.ppf(1-alpha/2, df=n-1)
    h = tcrit * s/np.sqrt(n)
    return (m, m-h, m+h, n)

def ci_prop(successes, n, alpha=0.05):
    if n == 0:
        return (np.nan, np.nan, np.nan, n)
    p = successes/n
    z = stats.norm.ppf(1-alpha/2)
    h = z*np.sqrt(p*(p-1)/n) if False else z*np.sqrt(p*(1-p)/n)
    return (p, max(0,p-h), min(1,p+h), n)

ics = {}
if 'Total' in v_orders.columns:
    ics['ticket'] = ci_mean(v_orders['Total'])
if 'delivery_delay_days' in v_orders.columns:
    ics['delay'] = ci_mean(v_orders['delivery_delay_days'])
if 'is_late' in v_orders.columns:
    p = v_orders['is_late'].mean()
    ics['late_prop'] = ci_prop(int(v_orders['is_late'].sum()), v_orders['is_late'].notna().sum())
if 'Purchase_Status' in v_orders.columns:
    is_cancel = (v_orders['Purchase_Status']=='Cancelado')
    ics['cancel_prop'] = ci_prop(int(is_cancel.sum()), is_cancel.notna().sum())
ics

{'ticket': (np.float64(2399.7437600000003),
  np.float64(2282.4101763045956),
  np.float64(2517.077343695405),
  500),
 'delay': (np.float64(0.446),
  np.float64(0.3071817212078458),
  np.float64(0.5848182787921542),
  500),
 'late_prop': (np.float64(0.422),
  np.float64(0.37871043382734293),
  np.float64(0.46528956617265704),
  np.int64(500)),
 'cancel_prop': (np.float64(0.114),
  np.float64(0.0861431262800224),
  np.float64(0.1418568737199776),
  np.int64(500))}

## 5) Testes de hipótese (≥ 10)

In [10]:
results = []

def add_result(name, H0, H1, stat, p, method, note=''):
    results.append({'teste': name, 'H0': H0, 'H1': H1, 'estatistica': stat, 'p_valor': p, 'metodo': method, 'nota': note})

# 1) ANOVA: Ticket ~ payment
if {'Total','payment'}.issubset(v_orders.columns):
    groups = [g['Total'].dropna() for _, g in v_orders.groupby('payment') if len(g['Total'].dropna())>=2]
    if len(groups) >= 2:
        stat, p = stats.f_oneway(*groups)
        add_result('ANOVA Ticket ~ payment',
                   'médias de Total iguais entre payments',
                   'ao menos um payment difere',
                   stat, p, 'ANOVA (one-way)')

# 2) Qui-quadrado: Cancelamento x payment
if {'Purchase_Status','payment'}.issubset(v_orders.columns):
    ct = pd.crosstab(v_orders['Purchase_Status'], v_orders['payment'])
    if ct.shape[0]>1 and ct.shape[1]>1:
        stat, p, dof, exp = stats.chi2_contingency(ct)
        add_result('Cancelamento x payment',
                   'independência entre status e payment',
                   'associação entre status e payment',
                   stat, p, 'Qui-quadrado de independência')

# 3) Proporção de atraso por Services
if {'is_late','Services'}.issubset(v_orders.columns):
    ct = pd.crosstab(v_orders['is_late'], v_orders['Services'])
    if ct.shape[0]>1 and ct.shape[1]>1:
        stat, p, dof, exp = stats.chi2_contingency(ct)
        add_result('Atraso x Services',
                   'proporção de atraso igual entre serviços',
                   'difere entre serviços',
                   stat, p, 'Qui-quadrado (multi-grupo)')

# 4) Lead time: Standard vs Same-Day (Welch t-test)
if {'delivery_lead_time','Services'}.issubset(v_orders.columns):
    a = v_orders.loc[v_orders['Services']=='Standard','delivery_lead_time'].dropna()
    b = v_orders.loc[v_orders['Services']=='Same-Day','delivery_lead_time'].dropna()
    if len(a)>=2 and len(b)>=2:
        stat, p = stats.ttest_ind(a, b, equal_var=False)
        add_result('Lead time Standard vs Same-Day',
                   'médias iguais',
                   'médias diferentes',
                   stat, p, 't-test Welch')

# 5) Correlação Desconto x Total (Spearman)
if {'Discount','Total'}.issubset(v_orders.columns):
    x = v_orders[['Discount','Total']].dropna()
    if len(x)>=3:
        rho, p = stats.spearmanr(x['Discount'], x['Total'])
        add_result('Desconto x Ticket',
                   'rho = 0',
                   'rho != 0',
                   rho, p, 'Spearman')

# 6) ANOVA: Category impacta Ticket
if {'Category','Total'}.issubset(v_orders.columns):
    groups = [g['Total'].dropna() for _, g in v_orders.groupby('Category') if len(g['Total'].dropna())>=2]
    if len(groups)>=2:
        stat, p = stats.f_oneway(*groups)
        add_result('Ticket ~ Category',
                   'médias iguais',
                   'ao menos uma categoria difere',
                   stat, p, 'ANOVA')

# 7) Região afeta taxa de atraso (Qui-quadrado)
if {'is_late','Region'}.issubset(v_orders.columns):
    ct = pd.crosstab(v_orders['is_late'], v_orders['Region'])
    if ct.shape[0]>1 and ct.shape[1]>1:
        stat, p, dof, exp = stats.chi2_contingency(ct)
        add_result('Atraso x Região',
                   'independência',
                   'dependência',
                   stat, p, 'Qui-quadrado')

# 8) Normalidade do Ticket (Shapiro)
if 'Total' in v_orders.columns and v_orders['Total'].notna().sum()>=3:
    sample = v_orders['Total'].dropna().sample(min(5000, v_orders['Total'].notna().sum()), random_state=42)
    stat, p = stats.shapiro(sample)
    add_result('Normalidade do Ticket',
               'distribuição normal',
               'não-normal',
               stat, p, 'Shapiro–Wilk (amostra)' )

# 9) Pedidos/dia ~ Poisson?
if 'Order_Date' in v_orders.columns:
    daily = v_orders.assign(day=pd.to_datetime(v_orders['Order_Date']).dt.date).groupby('day').size()
    lam = daily.mean() if len(daily)>0 else np.nan
    # Heurística: média ~ variância
    var = daily.var(ddof=1) if len(daily)>1 else np.nan
    # Qui-quadrado de aderência simplificado via bins (se suficiente)
    add_result('Contagem diária ~ Poisson?',
               'segue Poisson(λ)',
               'não segue Poisson',
               0 if not len(daily) else (var - lam),
               np.nan,
               'Diagnóstico: média≈variância; QQ-plot recomendado')

# 10) Atraso médio > 0? (t unilateral)
if 'delivery_delay_days' in v_orders.columns:
    x = v_orders['delivery_delay_days'].dropna().astype(float)
    if len(x)>=2:
        m = x.mean(); s = x.std(ddof=1); n = len(x)
        t = (m - 0)/(s/np.sqrt(n)) if s>0 else np.inf
        p = 1 - stats.t.cdf(t, df=n-1) if np.isfinite(t) else 0.0
        add_result('Atraso médio > 0?',
                   'μ ≤ 0',
                   'μ > 0',
                   t, p, 't-test unilateral')

# 11) Boleto tem maior cancelamento que PIX? (prop)
if {'Purchase_Status','payment'}.issubset(v_orders.columns):
    def prop(group):
        return (group=='Cancelado').mean(), group.notna().sum()
    g_boleto = v_orders.loc[v_orders['payment']=='Boleto','Purchase_Status']
    g_pix = v_orders.loc[v_orders['payment']=='PIX','Purchase_Status']
    if len(g_boleto)>0 and len(g_pix)>0:
        p1, n1 = prop(g_boleto)
        p2, n2 = prop(g_pix)
        # z para diferença unilateral p1 - p2 > 0
        p_pool = (p1*n1 + p2*n2) / (n1+n2) if (n1+n2)>0 else np.nan
        se = math.sqrt(p_pool*(1-p_pool)*(1/n1 + 1/n2)) if n1>0 and n2>0 else np.nan
        z = (p1 - p2)/se if se and se>0 else np.nan
        p = 1 - stats.norm.cdf(z) if np.isfinite(z) else np.nan
        add_result('Cancelamento: Boleto > PIX?',
                   'p_boleto - p_pix = 0',
                   '> 0',
                   z, p, 'z-proporções unilateral')

# 12) Freight por Services (ANOVA)
if {'P_Sevice','Services'}.issubset(v_orders.columns):
    groups = [g['P_Sevice'].dropna() for _, g in v_orders.groupby('Services') if len(g['P_Sevice'].dropna())>=2]
    if len(groups)>=2:
        stat, p = stats.f_oneway(*groups)
        add_result('Frete ~ Services',
                   'média igual',
                   'difere',
                   stat, p, 'ANOVA')

results_df = pd.DataFrame(results).sort_values('p_valor', na_position='last')
results_df.to_csv(os.path.join(BASE, 'figures', 'tests_results.csv'), index=False)
results_df.head(20)

,teste,H0,H1,estatistica,p_valor,metodo,nota
9,Frete ~ Services,média igual,difere,48.554816,5.479163e-20,ANOVA,
5,Normalidade do Ticket,distribuição normal,não-normal,0.960033,2.098058e-10,Shapiro–Wilk (amostra),
7,Atraso médio > 0?,μ ≤ 0,μ > 0,6.312348,3.038346e-10,t-test unilateral,
4,Desconto x Ticket,rho = 0,rho != 0,-0.128747,3.930748e-03,Spearman,
2,Atraso x Services,proporção de atraso igual entre serviços,difere entre serviços,2.973654,2.260889e-01,Qui-quadrado (multi-grupo),
3,Lead time Standard vs Same-Day,médias iguais,médias diferentes,-1.075321,2.829977e-01,t-test Welch,
8,Cancelamento: Boleto > PIX?,p_boleto - p_pix = 0,> 0,-0.503316,6.926291e-01,z-proporções unilateral,
1,Cancelamento x payment,independência entre status e payment,associação entre status e payment,1.043635,7.906951e-01,Qui-quadrado de independência,
0,ANOVA Ticket ~ payment,médias de Total iguais entre payments,ao menos um payment difere,0.191837,9.019595e-01,ANOVA (one-way),
6,Contagem diária ~ Poisson?,segue Poisson(λ),não segue Poisson,-0.821035,NaN,Diagnóstico: média≈variância; QQ-plot recomendado,


## 6) Modelagem leve

In [11]:
models = {}

# Regressão linear: Total ~ Discount + Services + Region + Category
required = {'Total','Discount','Services','Customer_Id'}
if required.issubset(v_orders.columns):
    df = v_orders.merge(con.execute('SELECT Customer_Id, Region FROM DIM_Customer').df(), on='Customer_Id', how='left')
    for col in ['Services','Region'] + ([]):
        if col in df.columns:
            df[col] = df[col].astype('category')
    formula = 'Total ~ Discount + C(Services) + C(Region)'
    try:
        m1 = smf.ols(formula, data=df.dropna(subset=['Total','Discount'])).fit()
        models['ols_total'] = m1.summary().as_text()
    except Exception as e:
        models['ols_total'] = str(e)

# Logística: is_late ~ Services + Region + Discount
req2 = {'is_late','Services','Customer_Id','Discount'}
if req2.issubset(v_orders.columns):
    df = v_orders.merge(con.execute('SELECT Customer_Id, Region FROM DIM_Customer').df(), on='Customer_Id', how='left')
    df = df.dropna(subset=['is_late']).copy()
    df['is_late'] = df['is_late'].astype(int)
    try:
        m2 = smf.logit('is_late ~ Discount + C(Services) + C(Region)', data=df).fit(disp=False)
        models['logit_late'] = m2.summary().as_text()
    except Exception as e:
        models['logit_late'] = str(e)

models


{'ols_total': '                            OLS Regression Results                            \n==============================================================================\nDep. Variable:                  Total   R-squared:                       0.023\nModel:                            OLS   Adj. R-squared:                  0.009\nMethod:                 Least Squares   F-statistic:                     1.623\nDate:                Wed, 22 Oct 2025   Prob (F-statistic):              0.127\nTime:                        16:06:49   Log-Likelihood:                -4301.7\nNo. Observations:                 500   AIC:                             8619.\nDf Residuals:                     492   BIC:                             8653.\nDf Model:                           7                                         \nCovariance Type:            nonrobust                                         \n============================================================================================\n           

## 7) Sumário executivo (auto-esqueleto)

In [13]:
# Gera um esqueleto de sumário com alguns números (se disponíveis)
summary_lines = []

def fmt_ci(tag, trip):
    """
    Formata médias e intervalos de confiança de forma segura.
    trip deve ser uma tupla/lista com (m, lo, hi, [n])
    """
    if isinstance(trip, (tuple, list)) and len(trip) >= 3:
        try:
            m, lo, hi = trip[:3]
            n = trip[3] if len(trip) > 3 else None
            return f"{tag}: média={m:,.2f}, IC95%=({lo:,.2f}, {hi:,.2f})" + (f", n={int(n)}" if n else "")
        except Exception:
            return None
    return None


# Monta as linhas do sumário
if 'ticket' in locals().get('ics', {}):
    summary_lines.append(fmt_ci("Ticket médio", ics['ticket']))

if 'delay' in locals().get('ics', {}):
    summary_lines.append(fmt_ci("Atraso (dias)", ics['delay']))

if 'late_prop' in locals().get('ics', {}):
    p, lo, hi, n = ics['late_prop']
    summary_lines.append(f"Taxa de atraso: {p:.2%} (IC95%: {lo:.2%}–{hi:.2%}, n={int(n)})")

if 'cancel_prop' in locals().get('ics', {}):
    p, lo, hi, n = ics['cancel_prop']
    summary_lines.append(f"Taxa de cancelamento: {p:.2%} (IC95%: {lo:.2%}–{hi:.2%}, n={int(n)})")


# Gera texto final e salva
summary_text = "\n".join([s for s in summary_lines if s])
os.makedirs(os.path.join(BASE, 'report'), exist_ok=True)

with open(os.path.join(BASE, 'report', 'executive_summary_auto.txt'), 'w', encoding='utf-8') as f:
    f.write(summary_text if summary_text else "Resumo será preenchido após carregar dados.")

summary_text or "Resumo será preenchido após carregar dados."


'Ticket médio: média=2,399.74, IC95%=(2,282.41, 2,517.08), n=500\nAtraso (dias): média=0.45, IC95%=(0.31, 0.58), n=500\nTaxa de atraso: 42.20% (IC95%: 37.87%–46.53%, n=500)\nTaxa de cancelamento: 11.40% (IC95%: 8.61%–14.19%, n=500)'

## 8) Exportáveis

In [14]:
# Salve tabelas importantes
v_orders.head(50).to_csv(os.path.join(BASE, 'figures', 'sample_v_orders.csv'), index=False)

# Renderização do relatório final fica a partir do template Markdown no diretório /report
print("Arquivos gerados em:", BASE)

Arquivos gerados em: C:/ecommerce_analytics_project/ecommerce_analytics_project
